In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pre_process_2 import pre_processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LayerNormalization, Dense, LSTM, Bidirectional, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
import numpy as np
import keras
from keras import layers

In [3]:
X, y = pre_processing('Modified_Result_5m_1.csv','with_Indicator_5m_BTC_USDT.csv', 15)
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [5]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout,
    mlp_dropout,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=128,
    num_heads=16,
    ff_dim=4,
    num_transformer_blocks=10,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.2,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=5e-3),
    metrics=["sparse_categorical_accuracy"],
)

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Epoch 1/150
23/23 [==============================] - 77s 2s/step - loss: 6032.2485 - sparse_categorical_accuracy: 0.3258 - val_loss: 816.7405 - val_sparse_categorical_accuracy: 0.3399
Epoch 2/150
23/23 [==============================] - 37s 2s/step - loss: 1327.3871 - sparse_categorical_accuracy: 0.3223 - val_loss: 558.4981 - val_sparse_categorical_accuracy: 0.2949
Epoch 3/150
23/23 [==============================] - 35s 2s/step - loss: 155.6958 - sparse_categorical_accuracy: 0.3448 - val_loss: 3.6600 - val_sparse_categorical_accuracy: 0.3652
Epoch 4/150
23/23 [==============================] - 33s 1s/step - loss: 3.0549 - sparse_categorical_accuracy: 0.3251 - val_loss: 1.3208 - val_sparse_categorical_accuracy: 0.3652
Epoch 5/150
23/23 [==============================] - 57s 3s/step - loss: 1.3982 - sparse_categorical_accuracy: 0.3216 - val_loss: 1.1069 - val_sparse_categorical_accuracy: 0.3315
Epoch 6/150
23/23 [==============================] - 59s 3s/step - loss: 1.1041 - sparse_cate

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('Result_1h_1.csv')
unique_elements, counts = np.unique(data, return_counts=True)
print("Unique elements:", unique_elements)
print("Counts:", counts)

TypeError: '<' not supported between instances of 'str' and 'int'